In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter

In [3]:
pickle_path = "./train.pkl"

with open(pickle_path, 'rb') as f:
    data = pickle.load(f)

test_path = "./test_no_target.pkl"

with open(test_path, 'rb') as f:
    test_data = pickle.load(f)

In [4]:
print(f"data length: {len(data)}")
print(f"element type: {type(data[0])}")
print(f"song fragment: {data[0][0][:10]}")
print(f"song class: {data[0][1]}")

data length: 2939
element type: <class 'tuple'>
song fragment: [ -1.  -1.  -1.  -1. 144. 144. 144.  64.  67.   0.]
song class: 0


In [5]:
X, y = zip(*data)

## Rozkład klas

In [6]:
class_counts = Counter(y)
print("class counts:")
for key, value in class_counts.items():
    print(f"{key}: {value}")

class counts:
0: 1630
1: 478
2: 154
3: 441
4: 236


## Statystyki utworów

In [7]:
lengths = [len(song) for song in X]
print(f"average length: {np.mean(lengths)}")
print(f"max length: {np.max(lengths)}")
print(f"min length: {np.min(lengths)}")

average length: 436.50493365090165
max length: 6308
min length: 4


## Preprocessing

In [8]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
def prepare_data(data, test=False):
    if test:
        sequences = [torch.tensor(seq, dtype=torch.float32) for seq in data]
        labels = None
    else:
        sequences, labels = zip(*data)
        sequences = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
        labels = torch.tensor(labels, dtype=torch.long)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return sequences, labels, lengths

In [21]:
def collate_fn(batch):
    if len(batch[0]) == 3:
        sequences, labels, lengths = zip(*batch)
        padded = pad_sequence(sequences, batch_first=True)
        return padded.unsqueeze(-1), torch.stack(labels), torch.stack(lengths)
    else:
        sequences, lengths = zip(*batch)
        padded = pad_sequence(sequences, batch_first=True)
        return padded.unsqueeze(-1), torch.stack(lengths)

## Dataloadery

In [23]:
def create_dataloader(data, batch_size, test=False):
    if test:
        sequences, _, lengths = prepare_data(data, test=True)
        dataset = list(zip(sequences, lengths))
    else:
        sequences, labels, lengths = prepare_data(data, test=False)
        dataset = list(zip(sequences, labels, lengths))
    return DataLoader(dataset, batch_size=batch_size, shuffle=not test, collate_fn=collate_fn)

In [24]:
BATCH_SIZE = 64
train_loader = create_dataloader(train_data, BATCH_SIZE)
val_loader = create_dataloader(val_data, BATCH_SIZE)
test_loader = create_dataloader(test_data, BATCH_SIZE, test=True)

## Model

In [13]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dir_mult = 2 if bidirectional else 1

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size * self.dir_mult, out_size)

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * self.dir_mult, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * self.dir_mult, x.size(0), self.hidden_size).to(device)
        x_packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(x_packed, (h0, c0))
        if self.bidirectional:
            last_hidden = torch.cat((hn[-2], hn[-1]), dim=1)
        else:
            last_hidden = hn[-1]
        out = self.fc(last_hidden)
        return out

## Trening

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y, lengths in loader:
            x, y, lengths = x.to(device), y.to(device), lengths.to(device)
            outputs = model(x, lengths)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

In [16]:
def train(model, train_loader, val_loader, epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for x, y, lengths in tqdm(train_loader, desc=f"Epoch {epoch}"):
            x, y, lengths = x.to(device), y.to(device), lengths.to(device)
            optimizer.zero_grad()
            outputs = model(x, lengths)
            loss = loss_fn(outputs, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        val_acc = evaluate(model, val_loader)
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}, Val Accuracy: {val_acc:.4f}")
    torch.save(model.state_dict(), "lstm_classifier.pt")

In [26]:
def generate_predictions(model, test_loader, output_file="pred.csv"):
    model.eval()
    predictions = []
    with torch.no_grad():
        for x, lengths in test_loader:
            x, lengths = x.to(device), lengths.to(device)
            outputs = model(x, lengths)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            predictions.extend(preds)
    with open(output_file, "w", newline="") as f:
        writer = csv.writer(f)
        for pred in predictions:
            writer.writerow([pred])
    print(f"Predictions saved to {output_file}")

In [18]:
model = LSTMClassifier(input_size=1, hidden_size=128, num_layers=2, out_size=5, bidirectional=True).to(device)
train(model, train_loader, val_loader, epochs=100)

Epoch 1: 100%|█████████████████████████████████████████| 37/37 [00:08<00:00,  4.20it/s]


Epoch 1, Loss: 46.5515, Val Accuracy: 0.5697


Epoch 2: 100%|█████████████████████████████████████████| 37/37 [00:08<00:00,  4.35it/s]


Epoch 2, Loss: 41.1233, Val Accuracy: 0.6173


Epoch 3: 100%|█████████████████████████████████████████| 37/37 [00:08<00:00,  4.30it/s]


...

Epoch 99: 100%|████████████████████████████████████████| 37/37 [00:08<00:00,  4.46it/s]


Epoch 99, Loss: 0.0142, Val Accuracy: 0.7602


Epoch 100: 100%|███████████████████████████████████████| 37/37 [00:08<00:00,  4.27it/s]


Epoch 100, Loss: 0.0138, Val Accuracy: 0.7636


ValueError: not enough values to unpack (expected 3, got 2)

In [25]:
generate_predictions(model, test_loader)

Predictions saved to predictions.csv
